# Part II: Optimizing Workforce Scheduling using Particle Swarm Optimization (PSO)

This notebook solves the workforce scheduling problem using Particle Swarm Optimization (PSO). The objective is to minimize total labor cost while meeting production staffing requirements during peak and non-peak hours.

In [1]:
!pip install pyswarms
import numpy as np
import pandas as pd
from pyswarms.single.global_best import GlobalBestPSO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.9 MB/s eta 0:00:00


## Problem Setup

We are scheduling 5 full-time, 5 overtime, and 5 part-time operators.  
Each operator has an hourly wage. Overtime is paid at 1.5x full-time rate.  
Constraints:
- At least 4 hours total during peak from full-time + overtime
- At least 2 hours total from part-time during non-peak

The optimization variables represent hours worked by each operator. The objective is to minimize total labor cost.

In [2]:
# Cost parameters
full_time = np.array([30, 35, 40, 45, 50])                      # Full-time rates
overtime = 1.5 * full_time                                      # Overtime rates
part_time = np.array([25, 27, 29, 31, 33])                      # Part-time rates

# Cost vector for all 15 decision variables
cost_vector = np.concatenate([full_time, overtime, part_time])

## Objective Function

The objective function calculates total labor cost and adds penalties if constraints are not met.
- If peak-hour labor < 4 hours → penalty
- If non-peak part-time labor < 2 hours → penalty


In [3]:
def total_labor_cost(x):
    penalties = []
    for xi in x:
        xi = np.clip(xi, 0, None)  # Ensure non-negative hours
        peak_hours = np.sum(xi[0:5]) + np.sum(xi[5:10])  # Full-time + Overtime
        non_peak_hours = np.sum(xi[10:15])               # Part-time only

        penalty = 0
        if peak_hours < 4:
            penalty += (4 - peak_hours) * 1000
        if non_peak_hours < 2:
            penalty += (2 - non_peak_hours) * 1000

        cost = np.sum(cost_vector * xi)
        penalties.append(cost + penalty)
    return np.array(penalties)

In [4]:
# Set bounds and PSO parameters
lower_bounds = np.zeros(15)
upper_bounds = np.ones(15) * 8  # Assume max 8 working hours

options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7}

# Run PSO
optimizer = GlobalBestPSO(n_particles=30, dimensions=15, options=options, bounds=(lower_bounds, upper_bounds))
best_cost, best_position = optimizer.optimize(total_labor_cost, iters=100)

2025-04-20 17:37:20,712 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=1.16e+3
2025-04-20 17:37:20,979 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1160.3514067968133, best pos: [2.03958446 5.39198281 2.36103812 0.2927696  1.37809299 0.53950333
 1.07657644 1.40983673 1.16208006 1.62558296 2.5316811  2.56318881
 2.98561551 1.24773018 3.34593379]


## Results and Interpretation

The best labor schedule found by PSO meets all constraints and minimizes cost.  
Below is a breakdown of hours assigned and associated cost per operator.

In [5]:
categories = ['Full-time (x_i)', 'Overtime (x_io)', 'Part-time (x_jp)']
rates = np.concatenate([full_time, overtime, part_time])

df = pd.DataFrame({
    'Category': [categories[i // 5] for i in range(15)],
    'Hourly Rate': rates,
    'Hours Worked': best_position,
    'Cost': rates * best_position
})

total_peak_hours = np.sum(best_position[0:10])
total_non_peak_hours = np.sum(best_position[10:15])

print(f"Total Labor Cost: {best_cost:.2f} SAR")
print(f"Peak Hours Total: {total_peak_hours:.2f} hours (Required ≥ 4)")
print(f"Non-Peak Part-Time Hours: {total_non_peak_hours:.2f} hours (Required ≥ 2)")

df


Total Labor Cost: 1160.35 SAR
Peak Hours Total: 17.28 hours (Required ≥ 4)
Non-Peak Part-Time Hours: 12.67 hours (Required ≥ 2)


,Category,Hourly Rate,Hours Worked,Cost
0,Full-time (x_i),30.0,2.039584,61.187534
1,Full-time (x_i),35.0,5.391983,188.719398
2,Full-time (x_i),40.0,2.361038,94.441525
3,Full-time (x_i),45.0,0.292770,13.174632
4,Full-time (x_i),50.0,1.378093,68.904650
5,Overtime (x_io),45.0,0.539503,24.277650
6,Overtime (x_io),52.5,1.076576,56.520263
7,Overtime (x_io),60.0,1.409837,84.590204
8,Overtime (x_io),67.5,1.162080,78.440404
9,Overtime (x_io),75.0,1.625583,121.918722
